In [ ]:
##open ai

##load credentials 

from dotenv import load_dotenv
import os

load_dotenv('../cred.env')

os.environ['OPENAI_API_KEY'] = os.getenv('openai_api_keys')

#OPENAI_API_KEY= os.environ['OPENAI_API_KEY']



In [17]:
## write function to use openai instance and get findings



from openai import OpenAI

client = OpenAI()

prompt = 'write a 10 lines poem on moon and stars with each sentence on newline'

response = client.responses.create(

    model = 'gpt-4o',
    input= prompt

)

print(response.output[0].content[0].text)


In the velvet night, the moon does glide,  
A silver coin on a vast, dark tide.  
Stars like whispers, tender and bright,  
Scattered gems in the cloak of night.  

They dance and flicker in cosmic play,  
Guiding lost souls who’ve lost their way.  
Dreams are woven in their gentle glow,  
Secrets the heavens quietly know.  

Silent guardians of night’s embrace,  
Eternal beauty in endless space.  


In [21]:
## write with role and message 

from openai import OpenAI

client = OpenAI()

sys_msg = 'You are poem writer for toddler books'
usr_prompt = 'write a 10 lines poem on moon and stars with each sentence on newline'


reponse = client.chat.completions.create(

model = 'gpt-4o',
messages = [

    {'role':'system', 'content':sys_msg},
    {'role':'user','content':usr_prompt}

],

temperature= 1

)

print(reponse.choices[0].message.content)

The moon peeks out, a glowing face,  
Floating gently in the sky's embrace.  
Silver beams on a gentle flight,  
Guide the stars through the night.  

Twinkling dots in a velvet sea,  
Sparkling brightly, wild and free.  
Like diamonds in the dark so wide,  
They dance and play in cosmic tide.  

Moon and stars with magic gleam,  
Whisper dreams in sleep's sweet stream.  


In [22]:
## for image

from openai import OpenAI

client = OpenAI()

sys_msg = 'Write concise summary of image'
usr_prompt = 'write a 10 lines poem on moon and stars with each sentence on newline'
image_url ='https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI'

reponse = client.chat.completions.create(

model = 'gpt-4o',
messages = [

    {'role':'system', 'content':sys_msg},

    {'role':'user','content':[

        {'type':'image_url','image_url':{'url': image_url}}

    ]
    }

],

temperature= 1

)

print(reponse.choices[0].message.content)

A snow-covered mountain peak rises against a backdrop of a pastel-colored sky at sunrise or sunset. The scene is tranquil and picturesque, showcasing the beauty of nature.


1.	Write a Python function to upsert raw texts into Pinecone without LangChain.

In [ ]:
##Write a Python function to upsert raw texts into Pinecone without LangChain.

from openai import OpenAI
from openai_pinecone import OpenAIPinecone
from dot_env import load_dotenv

load_dotenv(dotenv_path="../cred.env")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

##create index 

pc = OpenAIPinecone(api_key=OPENAI_API_KEY, env='us-west1-gcp')

index = pc.create_index(
    name="test",
    dimension=1536,
    metric="cosine",
    pod_type="p1"


)

pc.upsert(
    namespace="test",
    text="hello world",
    id="1"
)

In [ ]:
##2.	Modify your function to include metadata (like filename, chunk index).


pc.upsert(
    namespace="test",
    text="hello world",
    id="1",
    metadata={"filename": "file1.txt", "chunk_index": 1}
)

In [ ]:
#3.	Implement a search function that returns the top 3 most similar texts.

index.query(
    namespace="test",
    query="hello world",
    top_k=3,
    include_values=True,
    include_metadata=True
)

In [ ]:
## 5.	Show how to delete or update vectors in an existing Pinecone index.


index.delete(id="1", namespace="test") ## here namespace is mandatory

index.update(
    id="1",
    namespace="test",
    text="hello world 123",
    metadata={"filename": "file1.txt", "chunk_index": 1}
)

In [1]:
mys = ('abc',40)

print(type(mys))

<class 'tuple'>


In [3]:
sqr = lambda x=10: x ** 2

print(sqr(5))

print(sqr())

25
100
